In [30]:
import pandas as pd
import numpy as np


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

from sklearn.neighbors import LocalOutlierFactor

from DataMacpy import *

from joblib import dump, load

In [31]:
# Veriyi içeri aktaralım.

# veri yolumuzu verelim
file_path = 'train2.csv'

# verimizi okuyoruz. 
df = pd.read_csv(file_path)

print("Setup Complete")

Setup Complete


In [32]:
# target verilerimiz string olduğu için LabelEncoder yapıyoruz. Her sınıfı bir sayıya atıyoruz.
l_fit = LabelEncoder().fit(df["Öbek İsmi"])
y = l_fit.transform(df["Öbek İsmi"])
#y = pd.DataFrame(y, columns = ["Type_1"]) # dataframe'e çevirmek için bu satırı kullanabiliriz.
y[0:20]

array([3, 4, 7, 2, 2, 7, 2, 4, 2, 5, 4, 0, 6, 2, 6, 4, 6, 5, 5, 5])

In [33]:
# labellerimizi buluyoruz.
y_label = l_fit.inverse_transform([i for i in range(max(y)+1)])
label_and_num = list(zip(y_label, [i for i in range(max(y)+1)]))
label_and_num

[('obek_1', 0),
 ('obek_2', 1),
 ('obek_3', 2),
 ('obek_4', 3),
 ('obek_5', 4),
 ('obek_6', 5),
 ('obek_7', 6),
 ('obek_8', 7)]

In [34]:
df["Öbek İsmi"] = y

In [35]:
object_cols, d = calc_object_cols(df, 0)
d

{'Cinsiyet': 2,
 'Yaş Grubu': 5,
 'Medeni Durum': 2,
 'Eğitim Düzeyi': 9,
 'İstihdam Durumu': 4,
 'Yaşadığı Şehir': 4,
 'En Çok İlgilendiği Ürün Grubu': 5,
 'Eğitime Devam Etme Durumu': 2}

In [36]:
# Object kolonlarımıza OrdinalEncoder uyguluyoruz. OneHot denemesi yapılacak!
ordinal_encoder = OrdinalEncoder()
df[object_cols] = ordinal_encoder.fit_transform(df[object_cols])
df

,index,Cinsiyet,Yaş Grubu,Medeni Durum,Eğitim Düzeyi,İstihdam Durumu,Yıllık Ortalama Gelir,Yaşadığı Şehir,En Çok İlgilendiği Ürün Grubu,Yıllık Ortalama Satın Alım Miktarı,...,Eğitime Devam Etme Durumu,Öbek İsmi,Yıllık Ortalama Sepete Atılan Ürün Adedi,Yaş Durumu,Eğitim Durumu,Yaşadığı Şehir Durumu,Yıllık Ortalama Gelir Durumu,Yıllık Ortalama Satın Alım Durumu,Yıllık Ortalama Sipariş Verilen Ürün Adedi Durumu,Yıllık Ortalama Sepete Alınan Ürün Adedi Durumu
0,0,1.0,1.0,0.0,5.0,0.0,748266.440879,0.0,0.0,32010.900881,...,1.0,3,102.936904,1,3,2,4,3,3,4
1,1,0.0,4.0,1.0,3.0,2.0,246298.609605,2.0,4.0,4145.729891,...,1.0,4,82.176053,3,2,2,2,1,2,3
2,2,0.0,0.0,0.0,3.0,0.0,268582.732360,2.0,2.0,9924.379414,...,0.0,7,141.661426,1,2,2,2,2,3,4
3,3,0.0,3.0,1.0,3.0,2.0,327721.837865,3.0,2.0,6417.780714,...,1.0,2,17.097397,2,2,1,3,1,2,1
4,4,0.0,1.0,1.0,4.0,3.0,397431.626948,0.0,2.0,7886.663061,...,1.0,2,20.470459,1,1,2,3,1,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5455,5455,1.0,4.0,0.0,7.0,1.0,478054.247819,1.0,3.0,43182.299801,...,1.0,5,51.600436,3,3,1,3,3,2,2
5456,5456,0.0,3.0,1.0,1.0,2.0,481852.853651,1.0,1.0,42107.996193,...,1.0,5,49.190536,2,4,1,3,3,2,2
5457,5457,1.0,3.0,1.0,8.0,0.0,309443.687351,2.0,1.0,4655.392445,...,1.0,0,9.537384,2,1,2,3,1,2,1
5458,5458,1.0,0.0,0.0,6.0,0.0,266604.268016,0.0,1.0,9016.165906,...,0.0,7,143.281718,1,2,2,2,2,3,4


In [37]:
# özniteliklerimizi belirleyelim.
X = df.copy()
X.drop(["Öbek İsmi"], axis=1, inplace=True) # tüm verilerimizden hedef kolonumuzu çıkarıyoruz.
# Number kolonumuzu da gereksiz olduğu eğitimde gereksiz olduğu için çıkarıyoruz.
X

,index,Cinsiyet,Yaş Grubu,Medeni Durum,Eğitim Düzeyi,İstihdam Durumu,Yıllık Ortalama Gelir,Yaşadığı Şehir,En Çok İlgilendiği Ürün Grubu,Yıllık Ortalama Satın Alım Miktarı,Yıllık Ortalama Sipariş Verilen Ürün Adedi,Eğitime Devam Etme Durumu,Yıllık Ortalama Sepete Atılan Ürün Adedi,Yaş Durumu,Eğitim Durumu,Yaşadığı Şehir Durumu,Yıllık Ortalama Gelir Durumu,Yıllık Ortalama Satın Alım Durumu,Yıllık Ortalama Sipariş Verilen Ürün Adedi Durumu,Yıllık Ortalama Sepete Alınan Ürün Adedi Durumu
0,0,1.0,1.0,0.0,5.0,0.0,748266.440879,0.0,0.0,32010.900881,37.310644,1.0,102.936904,1,3,2,4,3,3,4
1,1,0.0,4.0,1.0,3.0,2.0,246298.609605,2.0,4.0,4145.729891,11.554851,1.0,82.176053,3,2,2,2,1,2,3
2,2,0.0,0.0,0.0,3.0,0.0,268582.732360,2.0,2.0,9924.379414,55.353236,0.0,141.661426,1,2,2,2,2,3,4
3,3,0.0,3.0,1.0,3.0,2.0,327721.837865,3.0,2.0,6417.780714,15.417239,1.0,17.097397,2,2,1,3,1,2,1
4,4,0.0,1.0,1.0,4.0,3.0,397431.626948,0.0,2.0,7886.663061,14.689837,1.0,20.470459,1,1,2,3,1,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5455,5455,1.0,4.0,0.0,7.0,1.0,478054.247819,1.0,3.0,43182.299801,31.356284,1.0,51.600436,3,3,1,3,3,2,2
5456,5456,0.0,3.0,1.0,1.0,2.0,481852.853651,1.0,1.0,42107.996193,26.288620,1.0,49.190536,2,4,1,3,3,2,2
5457,5457,1.0,3.0,1.0,8.0,0.0,309443.687351,2.0,1.0,4655.392445,5.156906,1.0,9.537384,2,1,2,3,1,2,1
5458,5458,1.0,0.0,0.0,6.0,0.0,266604.268016,0.0,1.0,9016.165906,39.205399,0.0,143.281718,1,2,2,2,2,3,4


In [38]:
# Verilerimizi train ve validation olarak ayırıyoruz.
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.9, test_size=0.1, random_state=0)

In [39]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
import lightgbm as lgb
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

from sklearn.model_selection import GridSearchCV

from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings('ignore')

In [40]:
model_list = [
    
    LogisticRegression(multi_class='multinomial', random_state=1, solver="newton-cg"), # 0
    
    KNeighborsClassifier(n_neighbors=3), # 1
    
    DecisionTreeClassifier(), # 2
    
    RandomForestClassifier(), # 3
    
    ExtraTreesClassifier(), # 4
    
    LGBMClassifier(), # 7
    
]

fast_model_select(model_list, X, y)

Model 0 Accuracy: %83.3882783882784
Model 1 Accuracy: %79.7802197802198
Model 2 Accuracy: %72.06959706959707
Model 3 Accuracy: %95.45787545787546
Model 4 Accuracy: %95.36630036630036
Model 5 Accuracy: %93.003663003663


In [41]:
params = {
    "n_estimators" : [75, 100, 125, 150, 200, 250, 500, 650, 1000],
    "min_samples_split" : [1, 3, 5, 7, 10, 15, 20]
}

In [42]:
extc_gscv_model = GridSearchCV( 
    ExtraTreesClassifier(),
    params,
    cv = 5,
    n_jobs = -1,
    verbose = 2).fit(X, y)

extc_gscv_model.best_params_

Fitting 5 folds for each of 63 candidates, totalling 315 fits


{'min_samples_split': 3, 'n_estimators': 100}

In [43]:
rf_gscv_model = GridSearchCV( 
    RandomForestClassifier(),
    params,
    cv = 5,
    n_jobs = -1,
    verbose = 2).fit(X, y)

rf_gscv_model.best_params_

Fitting 5 folds for each of 63 candidates, totalling 315 fits


{'min_samples_split': 3, 'n_estimators': 500}

In [65]:
model_list2 = [
    
    ExtraTreesClassifier(**extc_gscv_model.best_params_),
    
    ExtraTreesClassifier(n_estimators=2000),
    
    ExtraTreesClassifier(),
    
    RandomForestClassifier(**rf_gscv_model.best_params_),
    
    RandomForestClassifier(),    
    
]

fast_model_select(model_list2, X, y)

Model 0 Accuracy: %95.47619047619048
Model 1 Accuracy: %95.43956043956044
Model 2 Accuracy: %95.45787545787546
Model 3 Accuracy: %95.47619047619048
Model 4 Accuracy: %95.47619047619048
Model 5 Accuracy: %94.74358974358972


In [71]:
def fast_model_select2(model_list, X, y):
    """
    Liste şeklinde kıyaslama yapacağımız modellerimizi ve tüm verilerimizi alarak 3 çapraz doğrulama oluşturup
    modellerin doğruluklarını döndürür.
    """
    
    score_list = []
    
    for i, model in enumerate(model_list):
        scores = cross_val_score(model, X, y, scoring='accuracy', cv=3)
        print(f"Model {i} Accuracy: %{scores.mean()*100}")
        
        score_list.append(scores.mean()*100)
        
    score_list.sort()    
    return score_list[-1]

In [73]:
model_list2 = [
    
    ExtraTreesClassifier(**extc_gscv_model.best_params_),
    
    ExtraTreesClassifier(),
    
    RandomForestClassifier(**rf_gscv_model.best_params_),
    
    RandomForestClassifier(),
         
]

extc_gscv_model.best_params_ = {'min_samples_split': 3, 'n_estimators': 100}

model = ExtraTreesClassifier(**extc_gscv_model.best_params_)

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.85, test_size=0.15, random_state=69)

model.fit(X_train, y_train)


for i in range(0,100):
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.85, test_size=0.15, random_state=i)
    print("#"*10," "*10,i," "*10,"#"*10)
    print("\n")
    print("----->>>>>",fast_model_select2(model_list2, X_train, y_train),"<<<<<-----")
    print("\n")

##########            0            ##########
Model 0 Accuracy: %95.45356604180134
Model 1 Accuracy: %95.43201896143073
Model 2 Accuracy: %95.43201896143073
Model 3 Accuracy: %95.45356604180134
----->>>>> 95.45356604180134 <<<<<-----


##########            1            ##########
Model 0 Accuracy: %95.49666020254256
Model 1 Accuracy: %95.41047188106012
Model 2 Accuracy: %95.51820728291317
Model 3 Accuracy: %95.45356604180134
----->>>>> 95.51820728291317 <<<<<-----


##########            2            ##########
Model 0 Accuracy: %95.21654815772463
Model 1 Accuracy: %95.25964231846585
Model 2 Accuracy: %95.23809523809524
Model 3 Accuracy: %95.23809523809524
----->>>>> 95.25964231846585 <<<<<-----


##########            3            ##########
Model 0 Accuracy: %95.49666020254254
Model 1 Accuracy: %95.51820728291315
Model 2 Accuracy: %95.49666020254254
Model 3 Accuracy: %95.49666020254254
----->>>>> 95.51820728291315 <<<<<-----


##########            4            ##########
Model 0 Ac

Model 0 Accuracy: %95.34583063994829
Model 1 Accuracy: %95.34583063994829
Model 2 Accuracy: %95.38892480068951
Model 3 Accuracy: %95.38892480068951
----->>>>> 95.38892480068951 <<<<<-----


##########            36            ##########
Model 0 Accuracy: %95.51820728291317
Model 1 Accuracy: %95.49666020254256
Model 2 Accuracy: %95.53975436328378
Model 3 Accuracy: %95.51820728291317
----->>>>> 95.53975436328378 <<<<<-----


##########            37            ##########
Model 0 Accuracy: %95.19500107735401
Model 1 Accuracy: %95.19500107735401
Model 2 Accuracy: %95.19500107735402
Model 3 Accuracy: %95.21654815772463
----->>>>> 95.21654815772463 <<<<<-----


##########            38            ##########
Model 0 Accuracy: %95.38892480068951
Model 1 Accuracy: %95.38892480068951
Model 2 Accuracy: %95.41047188106012
Model 3 Accuracy: %95.38892480068951
----->>>>> 95.41047188106012 <<<<<-----


##########            39            ##########
Model 0 Accuracy: %95.34583063994829
Model 1 Accurac

Model 0 Accuracy: %95.49666020254256
Model 1 Accuracy: %95.41047188106012
Model 2 Accuracy: %95.47511312217195
Model 3 Accuracy: %95.47511312217195
----->>>>> 95.49666020254256 <<<<<-----


##########            71            ##########
Model 0 Accuracy: %95.19500107735401
Model 1 Accuracy: %95.1734539969834
Model 2 Accuracy: %95.28118939883646
Model 3 Accuracy: %95.23809523809524
----->>>>> 95.28118939883646 <<<<<-----


##########            72            ##########
Model 0 Accuracy: %95.41047188106012
Model 1 Accuracy: %95.3673777203189
Model 2 Accuracy: %95.3889248006895
Model 3 Accuracy: %95.36737772031888
----->>>>> 95.41047188106012 <<<<<-----


##########            73            ##########
Model 0 Accuracy: %95.47511312217195
Model 1 Accuracy: %95.43201896143073
Model 2 Accuracy: %95.45356604180134
Model 3 Accuracy: %95.45356604180134
----->>>>> 95.47511312217195 <<<<<-----


##########            74            ##########
Model 0 Accuracy: %95.3673777203189
Model 1 Accuracy: %

In [55]:
model = RandomForestClassifier(**rf_gscv_model.best_params_).fit(X, y)

In [74]:
extc_gscv_model.best_params_ = {'min_samples_split': 3, 'n_estimators': 100}

model = ExtraTreesClassifier(**extc_gscv_model.best_params_)

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.85, test_size=0.15, random_state=69)

model.fit(X_train, y_train)

ExtraTreesClassifier(min_samples_split=3)

## Submisson

In [75]:
# Veriyi içeri aktaralım.

# veri yolumuzu verelim
file_path2 = 'test_x2.csv'

# verimizi okuyoruz. 
df2 = pd.read_csv(file_path2)

print("Setup Complete")

Setup Complete


In [76]:
df2

,index,Cinsiyet,Yaş Grubu,Medeni Durum,Eğitim Düzeyi,İstihdam Durumu,Yıllık Ortalama Gelir,Yaşadığı Şehir,En Çok İlgilendiği Ürün Grubu,Yıllık Ortalama Satın Alım Miktarı,Yıllık Ortalama Sipariş Verilen Ürün Adedi,Eğitime Devam Etme Durumu,Yıllık Ortalama Sepete Atılan Ürün Adedi,Yaş Durumu,Eğitim Durumu,Yaşadığı Şehir Durumu,Yıllık Ortalama Gelir Durumu,Yıllık Ortalama Satın Alım Durumu,Yıllık Ortalama Sipariş Verilen Ürün Adedi Durumu,Yıllık Ortalama Sepete Alınan Ürün Adedi Durumu
0,0,Erkek,51-60,Evli,Yüksek Lisans Mezunu,Düzenli ve Ücretli Bir İşi Var,365331.930346,Köy veya Kasaba,Giyim,5566.587664,15.442635,Etmiyor,19.646460,2,3,1,3,1,2,1
1,1,Erkek,41-50,Bekar,Lise Mezunu,İşsiz veya Düzenli Bir İşi Yok,414899.236174,Büyük Şehir,Spor Malzemeleri,6756.766203,14.957246,Etmiyor,19.675863,2,2,2,3,1,2,1
2,2,Kadın,>60,Evli,Eğitimsiz,İşsiz veya Düzenli Bir İşi Yok,152825.872271,Küçük Şehir,Giyim,4493.915682,10.907930,Etmiyor,46.904518,3,1,2,2,1,2,2
3,3,Kadın,51-60,Evli,Doktora Ötesi,Düzenli ve Ücretli Bir İşi Var,489052.908472,Köy veya Kasaba,Kozmetik,42670.646595,32.333021,Etmiyor,48.083609,2,4,1,3,3,2,2
4,4,Erkek,18-30,Evli,Yüksek Lisans Mezunu,Kendi İşinin Sahibi,323635.711534,Küçük Şehir,Giyim,5959.266948,4.519968,Etmiyor,12.752491,1,3,2,3,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2335,2335,Kadın,18-30,Bekar,Yüksekokul Mezunu,Kendi İşinin Sahibi,259190.078834,Küçük Şehir,Ev ve Mobilya,9559.156534,41.883661,Ediyor,117.627785,1,2,2,2,2,3,4
2336,2336,Kadın,31-40,Evli,Lise Mezunu,İşsiz veya Düzenli Bir İşi Yok,194580.563177,Kırsal,Ev ve Mobilya,4070.232154,20.082996,Etmiyor,45.659622,1,2,1,2,1,2,2
2337,2337,Erkek,>60,Evli,Yüksek Lisans Mezunu,Düzenli ve Ücretli Bir İşi Var,357730.521969,Büyük Şehir,Ev ve Mobilya,22589.204381,18.922305,Etmiyor,100.691379,3,3,2,3,3,2,4
2338,2338,Erkek,18-30,Evli,Doktora Ötesi,Düzenli ve Ücretli Bir İşi Var,213179.637202,Küçük Şehir,Spor Malzemeleri,4205.390562,10.758108,Etmiyor,97.778913,1,4,2,2,1,2,3


In [77]:
object_cols, d = calc_object_cols(df2, 0)
d

{'Cinsiyet': 2,
 'Yaş Grubu': 5,
 'Medeni Durum': 2,
 'Eğitim Düzeyi': 9,
 'İstihdam Durumu': 4,
 'Yaşadığı Şehir': 4,
 'En Çok İlgilendiği Ürün Grubu': 5,
 'Eğitime Devam Etme Durumu': 2}

In [78]:
# Object kolonlarımıza OrdinalEncoder uyguluyoruz. OneHot denemesi yapılacak!
ordinal_encoder = OrdinalEncoder()
df2[object_cols] = ordinal_encoder.fit_transform(df2[object_cols])
df2

,index,Cinsiyet,Yaş Grubu,Medeni Durum,Eğitim Düzeyi,İstihdam Durumu,Yıllık Ortalama Gelir,Yaşadığı Şehir,En Çok İlgilendiği Ürün Grubu,Yıllık Ortalama Satın Alım Miktarı,Yıllık Ortalama Sipariş Verilen Ürün Adedi,Eğitime Devam Etme Durumu,Yıllık Ortalama Sepete Atılan Ürün Adedi,Yaş Durumu,Eğitim Durumu,Yaşadığı Şehir Durumu,Yıllık Ortalama Gelir Durumu,Yıllık Ortalama Satın Alım Durumu,Yıllık Ortalama Sipariş Verilen Ürün Adedi Durumu,Yıllık Ortalama Sepete Alınan Ürün Adedi Durumu
0,0,0.0,3.0,1.0,5.0,0.0,365331.930346,1.0,2.0,5566.587664,15.442635,1.0,19.646460,2,3,1,3,1,2,1
1,1,0.0,2.0,0.0,3.0,3.0,414899.236174,0.0,4.0,6756.766203,14.957246,1.0,19.675863,2,2,2,3,1,2,1
2,2,1.0,4.0,1.0,2.0,3.0,152825.872271,2.0,2.0,4493.915682,10.907930,1.0,46.904518,3,1,2,2,1,2,2
3,3,1.0,3.0,1.0,1.0,0.0,489052.908472,1.0,3.0,42670.646595,32.333021,1.0,48.083609,2,4,1,3,3,2,2
4,4,0.0,0.0,1.0,5.0,2.0,323635.711534,2.0,2.0,5959.266948,4.519968,1.0,12.752491,1,3,2,3,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2335,2335,1.0,0.0,0.0,6.0,2.0,259190.078834,2.0,1.0,9559.156534,41.883661,0.0,117.627785,1,2,2,2,2,3,4
2336,2336,1.0,1.0,1.0,3.0,3.0,194580.563177,3.0,1.0,4070.232154,20.082996,1.0,45.659622,1,2,1,2,1,2,2
2337,2337,0.0,4.0,1.0,5.0,0.0,357730.521969,0.0,1.0,22589.204381,18.922305,1.0,100.691379,3,3,2,3,3,2,4
2338,2338,0.0,0.0,1.0,1.0,0.0,213179.637202,2.0,4.0,4205.390562,10.758108,1.0,97.778913,1,4,2,2,1,2,3


In [79]:
preds = model.predict(df2)
preds

array([2, 2, 1, ..., 6, 4, 5])

In [80]:
# labellerimizi buluyoruz.
preds_labelled = l_fit.inverse_transform([i for i in preds])
preds_labelled

array(['obek_3', 'obek_3', 'obek_2', ..., 'obek_7', 'obek_5', 'obek_6'],
      dtype=object)

In [81]:
sub = pd.DataFrame(data=preds_labelled, columns=["Öbek İsmi"])
sub

,Öbek İsmi
0,obek_3
1,obek_3
2,obek_2
3,obek_6
4,obek_1
...,...
2335,obek_8
2336,obek_2
2337,obek_7
2338,obek_5


In [82]:
sub = sub.reset_index()
sub = sub.rename(columns={"index": "id"})
sub

,id,Öbek İsmi
0,0,obek_3
1,1,obek_3
2,2,obek_2
3,3,obek_6
4,4,obek_1
...,...,...
2335,2335,obek_8
2336,2336,obek_2
2337,2337,obek_7
2338,2338,obek_5


In [83]:
sub.to_csv('last_dance.csv', index=False, encoding='utf-8')